In [22]:
import requests
from bs4 import BeautifulSoup
import csv
import time

In [23]:
def scrape_reddit_html(url):
    headers = {"User-Agent": "Mozilla/5.0"}
    print("Fetching:", url)

    r = requests.get(url, headers=headers, timeout=30)
    soup = BeautifulSoup(r.text, "html.parser")

    comments = []

    for div in soup.select("div.comment"):
        body = div.select_one(".md")
        author = div.select_one(".author")

        if body:
            text = body.get_text(" ", strip=True)
            text = text.replace("\n", " ").replace("\r", " ").strip()
        else:
            text = ""

        comments.append({
            "author": author.get_text(strip=True) if author else "[deleted]",
            "body": text
        })

    return comments


def scrape_full_thread(base_url):
    all_comments = []

    url = base_url
    while True:
        comments = scrape_reddit_html(url)
        if not comments:
            break

        all_comments.extend(comments)

        # cek ada next button?
        soup = BeautifulSoup(requests.get(url, headers={"User-Agent":"Mozilla/5.0"}).text, "html.parser")
        next_button = soup.select_one("span.next-button a")

        if next_button:
            url = next_button["href"]
            print("Next page →", url)
            time.sleep(1)
        else:
            break

    return all_comments

In [24]:
def save_csv(data, filename):
    with open(filename, "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=["author", "body"])
        writer.writeheader()
        writer.writerows(data)

    print("Saved →", filename)

In [25]:
# -------------------------------------
# RUN
# -------------------------------------
url = "https://old.reddit.com/r/AskReddit/comments/dkjimg/what_screams_im_very_insecure/?sort=best"

comments = scrape_full_thread(url)
save_csv(comments, "what_screams_im_very_insecure_FULL.csv")

print("Total comments scraped:", len(comments))

Fetching: https://old.reddit.com/r/AskReddit/comments/dkjimg/what_screams_im_very_insecure/?sort=best


SSLError: HTTPSConnectionPool(host='old.reddit.com', port=443): Max retries exceeded with url: /r/AskReddit/comments/dkjimg/what_screams_im_very_insecure/?sort=best (Caused by SSLError(SSLCertVerificationError(1, "[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: Hostname mismatch, certificate is not valid for 'old.reddit.com'. (_ssl.c:1017)")))